## Import Libraries

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import nltk
import pandas
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding, Dense
from keras.layers import TimeDistributed, LSTM
from keras.engine.input_layer import Input
from sklearn.model_selection import train_test_split
import wget
import zipfile
import os, fnmatch
import seaborn as sns
import pickle as pkl
import random
import re
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


## Download and transform the data

In [2]:
# Download and extract the dataset
#def fetch_data(web_file, local_dir='.'):
#    """Download the `web_file`, assuming it is a web resource into the local_dir. 
#    If a file with the same filename already exists in the local directory, do not 
#    download it but return its path instead.
#    Arguments:
#        web_file: a web resource identifiable by a url (str)
#        local_dir: a local directory to download the web_file into (str)
#    Return: The local path to the file (str)
#    """
#    file_name = local_dir + "/" + web_file.rsplit("/",1)[-1]
#    if os.path.exists(file_name):
#        return file_name
#    else:
#        file_name = wget.download(web_file, out=local_dir)
#        return file_name
#data_filename = fetch_data('https://s3.amazonaws.com/pytorch-tutorial-assets/cornell_movie_dialogs_corpus.zip')
#with zipfile.ZipFile(data_filename, 'r') as zip_ref:
#    zip_ref.extractall('.\data')

In [3]:
# load the movie lines
#movie_lines_features = ["LineID", "Character", "Movie", "Name", "Line"]
#movie_lines = pd.read_csv('.\\data\\cornell movie-dialogs corpus\\movie_lines.txt', 
#                          engine = "python", 
#                          index_col = False,
#                          sep=' \+\+\+\$\+\+\+ ',
#                          names = movie_lines_features)
#
## Using only the required columns, namely, "LineID" and "Line"
#movie_lines = movie_lines[["LineID", "Line"]]
#
## Strip the space from "LineID" for further usage and change the datatype of "Line"
#movie_lines["LineID"] = movie_lines["LineID"].apply(str.strip)

In [4]:
#movie_lines.head()

In [5]:
# Load the conversations file
#movie_conversations_features = ["Character1", "Character2", "Movie", "Conversation"]
#movie_conversations = pd.read_csv('.\\data\\cornell movie-dialogs corpus\\movie_conversations.txt',
#                                  sep = "\+\+\+\$\+\+\+", 
#                                  engine = "python", 
#                                  index_col = False, 
#                                  names = movie_conversations_features)
#
# Again using the required feature, "Conversation"
# movie_conversations = movie_conversations["Conversation"]

# Preprocessing and storing the conversation data. This takes too long to run, so we saved the result as a pickle
# conversation = [[str(list(movie_lines.loc[movie_lines["LineID"] == u.strip().strip("'"), "Line"])[0]).strip() for u in c.strip().strip('[').strip(']').split(',')] for c in movie_conversations]

#with open(".\\data\\conversations.pkl", "wb") as handle:
    #pkl.dump(conversation, handle)

In [3]:
with open(r"data\conversations.pkl", "rb") as handle:
    conversation = pkl.load(handle)

In [4]:
sample_size = 30000

indices = random.sample(range(len(conversation)), sample_size)

conv_sample = []

for i in indices:
    conv = conversation[i]
    conv_sample.append(conv)


We now have a list of conversations, ready to use. These are the tasks we need to do: 
* Create pairs of questions and answers
* Text Cleaning
* Remove too large and too small utterances
* Put \<*BOS*> tag and \<*EOS*> tag for decoder input
* Make Vocabulary (VOCAB_SIZE)
* Tokenize Bag of words to Bag of IDs
* Padding (MAX_LEN)
* Word Embedding (EMBEDDING_DIM)
* Reshape the Data depends on neural network shape
* Split Data for training and validation, testing

In [5]:
# Hyperparams
min_line_length = 1
max_line_length = 18
VOCAB_SIZE= 10000
HIDDEN_DIM=300

In [6]:
# Sort the sentences into questions (inputs) and answers (targets)
questions = []
answers = []

for conv in conv_sample:
    for i in range(len(conv)-1):
        questions.append(conv[i])
        answers.append(conv[i+1])

In [7]:
# Check if we have loaded the data correctly
limit = 0
for i in range(limit, limit+5):
    print(questions[i])
    print(answers[i])
    print()

Here now, what are you fixin' to do?
Have a look.  At nothing.

You've got to tell Craig what's going on. He must never leave Malkovich.
I'm glad you learned sign language, Elijah, but I'm tired of your nagging. I'm tired of this conversation. I'm tired period. What has the world ever done for me that I should feel personally responsible for saving it?

I'm glad you learned sign language, Elijah, but I'm tired of your nagging. I'm tired of this conversation. I'm tired period. What has the world ever done for me that I should feel personally responsible for saving it?
It is better to light one candle than curse the darkness. I learned that from you.

Whoever hired us.
I got this gig through a contractor. And he most definitely didn't know who was doing the hiring, only that they were paying a lot of money...

It's all my fault, I'm so sorry Doug.
I had it coming from someone.



In [8]:
# Compare lengths of questions and answers
print(len(questions))
print(len(answers))

79766
79766


In [9]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = re.sub(r"  ", " ", text)
    return text

In [10]:
# Clean the data
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

In [11]:
# Take a look at some of the data to ensure that it has been cleaned well.
limit = 0
for i in range(limit, limit+5):
    print(clean_questions[i])
    print(clean_answers[i])
    print()

here now what are you fixing to do
have a look at nothing

you have got to tell craig that is going on he must never leave malkovich
i am glad you learned sign language elijah but i am tired of your nagging i am tired of this conversation i am tired period what has the world ever done for me that i should feel personally responsible for saving it

i am glad you learned sign language elijah but i am tired of your nagging i am tired of this conversation i am tired period what has the world ever done for me that i should feel personally responsible for saving it
it is better to light one candle than curse the darkness i learned that from you

whoever hired us
i got this gig through a contractor and he most definitely did not know who was doing the hiring only that they were paying a lot of money

it is all my fault i am so sorry doug
i had it coming from someone



In [12]:
# Find the length of sentences
lengths = []
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))

# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [13]:
lengths.describe()

,counts
count,159532.000000
mean,10.902245
std,12.190124
min,1.000000
25%,4.000000
50%,7.000000
75%,14.000000
max,365.000000


In [14]:
# Remove questions and answers that are shorter than <min_line_length> words and longer than <max_line_length> words.

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

i = 0
for question in clean_questions:
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])
    i += 1

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

i = 0
for answer in short_answers_temp:
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
    i += 1

In [15]:
# Compare the number of lines we will use with the total number of lines.
print("# of questions:", len(short_questions))
print("# of answers:", len(short_answers))
print("% of data used: {}%".format(round(len(short_questions)/len(questions),4)*100))

# of questions: 56571
# of answers: 56571
% of data used: 70.92%


In [16]:
# put <BOS> and <EOS> tags on what will be the decoder input

def tagger(decoder_input_sentence):
    bos = "<BOS> "
    eos = " <EOS>"
    final_target = [bos + text + eos for text in decoder_input_sentence] 
    return final_target

tagged_answers = tagger(short_answers)

In [17]:
# make vocabulary of arbitrary size

def vocab_creator(text_lists, VOCAB_SIZE):
    tokenizer = Tokenizer(num_words=VOCAB_SIZE)
    tokenizer.fit_on_texts(text_lists)
    dictionary = tokenizer.word_index
    word2idx = {}
    idx2word = {}
    for k, v in dictionary.items():
        if v < VOCAB_SIZE:
            word2idx[k] = v
            idx2word[v] = k
        if v >= VOCAB_SIZE-1:
            continue
          
    return word2idx, idx2word


word2idx, idx2word = vocab_creator(text_lists=short_questions+tagged_answers, VOCAB_SIZE=VOCAB_SIZE)

In [18]:
word2idx

{'bos': 1,
 'eos': 2,
 'you': 3,
 'i': 4,
 'is': 5,
 'the': 6,
 'not': 7,
 'to': 8,
 'it': 9,
 'a': 10,
 'do': 11,
 'that': 12,
 'are': 13,
 'what': 14,
 'me': 15,
 'have': 16,
 'of': 17,
 'and': 18,
 'we': 19,
 'in': 20,
 'am': 21,
 'he': 22,
 'no': 23,
 'this': 24,
 'will': 25,
 'your': 26,
 'know': 27,
 'for': 28,
 'was': 29,
 'my': 30,
 'be': 31,
 'on': 32,
 'did': 33,
 'just': 34,
 'about': 35,
 'they': 36,
 'like': 37,
 'would': 38,
 'with': 39,
 'how': 40,
 'get': 41,
 'all': 42,
 'here': 43,
 'but': 44,
 'she': 45,
 'so': 46,
 'yes': 47,
 'got': 48,
 'want': 49,
 'out': 50,
 'well': 51,
 'him': 52,
 'why': 53,
 'think': 54,
 'yeah': 55,
 'oh': 56,
 'can': 57,
 'right': 58,
 'up': 59,
 'go': 60,
 'there': 61,
 'if': 62,
 'going': 63,
 'at': 64,
 'one': 65,
 'where': 66,
 'her': 67,
 'now': 68,
 'see': 69,
 'good': 70,
 'cannot': 71,
 'who': 72,
 'come': 73,
 'tell': 74,
 'say': 75,
 'were': 76,
 'could': 77,
 'then': 78,
 'time': 79,
 'okay': 80,
 'back': 81,
 'from': 82,
 'been

In [19]:
idx2word

{1: 'bos',
 2: 'eos',
 3: 'you',
 4: 'i',
 5: 'is',
 6: 'the',
 7: 'not',
 8: 'to',
 9: 'it',
 10: 'a',
 11: 'do',
 12: 'that',
 13: 'are',
 14: 'what',
 15: 'me',
 16: 'have',
 17: 'of',
 18: 'and',
 19: 'we',
 20: 'in',
 21: 'am',
 22: 'he',
 23: 'no',
 24: 'this',
 25: 'will',
 26: 'your',
 27: 'know',
 28: 'for',
 29: 'was',
 30: 'my',
 31: 'be',
 32: 'on',
 33: 'did',
 34: 'just',
 35: 'about',
 36: 'they',
 37: 'like',
 38: 'would',
 39: 'with',
 40: 'how',
 41: 'get',
 42: 'all',
 43: 'here',
 44: 'but',
 45: 'she',
 46: 'so',
 47: 'yes',
 48: 'got',
 49: 'want',
 50: 'out',
 51: 'well',
 52: 'him',
 53: 'why',
 54: 'think',
 55: 'yeah',
 56: 'oh',
 57: 'can',
 58: 'right',
 59: 'up',
 60: 'go',
 61: 'there',
 62: 'if',
 63: 'going',
 64: 'at',
 65: 'one',
 66: 'where',
 67: 'her',
 68: 'now',
 69: 'see',
 70: 'good',
 71: 'cannot',
 72: 'who',
 73: 'come',
 74: 'tell',
 75: 'say',
 76: 'were',
 77: 'could',
 78: 'then',
 79: 'time',
 80: 'okay',
 81: 'back',
 82: 'from',
 83: '

In [20]:
# create an index for the padder 
idx2word[0]=' '
word2idx[' ']= 0


In [21]:
def text2seq(encoder_text, decoder_text, VOCAB_SIZE):

    tokenizer = Tokenizer(num_words=VOCAB_SIZE)
    tokenizer.fit_on_texts(encoder_text+decoder_text)
    encoder_sequences = tokenizer.texts_to_sequences(encoder_text)
    decoder_sequences = tokenizer.texts_to_sequences(decoder_text)
  
    return encoder_sequences, decoder_sequences

encoder_sequences, decoder_sequences = text2seq(short_questions, tagged_answers, VOCAB_SIZE) 

In [22]:
encoder_sequences[0:10]

[[43, 68, 14, 13, 3, 2145, 8, 11],
 [9, 5, 42, 30, 807, 4, 21, 46, 110, 1999],
 [4, 102, 9, 211, 82, 246],
 [192, 60, 39, 6, 2333, 61],
 [4, 302, 8, 60, 39, 6, 4682],
 [6, 4682, 53],
 [347, 19, 2611, 1329, 12, 5, 9, 3584, 5, 1247],
 [30, 442],
 [66, 11, 19, 288],
 [9, 5, 10, 181, 187, 40, 35, 6, 1302]]

In [23]:
decoder_sequences[0:10]

[[1, 16, 10, 87, 64, 121, 2],
 [1, 4, 102, 9, 211, 82, 246, 2],
 [1, 13, 3, 92, 4, 103, 7, 41, 10, 370, 2],
 [1, 4, 302, 8, 60, 39, 6, 4682, 2],
 [1, 6, 4682, 53, 2],
 [1, 689, 171, 52, 12, 5, 53, 2],
 [1, 24, 5, 378, 1132, 4, 21, 2591, 2],
 [1, 55, 12, 5, 988, 97, 2],
 [1, 9, 5, 10, 181, 187, 40, 35, 6, 1302, 2],
 [1, 875, 2]]

In [24]:
# the padding is 2 characters longer than max_line_length to account for the <BOS> and <EOS> tags

def padding(encoder_sequences, decoder_sequences, MAX_LEN):
  
    encoder_input_data = pad_sequences(encoder_sequences, maxlen=MAX_LEN, dtype='int32', padding='post', truncating='post')
    decoder_input_data = pad_sequences(decoder_sequences, maxlen=MAX_LEN, dtype='int32', padding='post', truncating='post')
    decoder_output_data = pad_sequences(decoder_sequences, maxlen=MAX_LEN+1, dtype='int32', padding='post', truncating='post')
    return encoder_input_data, decoder_input_data, decoder_output_data

encoder_padded_seq, decoder_padded_seq, decoder_padded_seq_1 = padding(encoder_sequences, decoder_sequences, MAX_LEN=max_line_length+2)


In [25]:
encoder_padded_seq.shape

(56571, 20)

In [26]:
decoder_padded_seq_1[:,1:].shape

(56571, 20)

In [27]:
decoder_padded_seq.shape

(56571, 20)

In [35]:
encoder_input_data = encoder_padded_seq
decoder_input_data = decoder_padded_seq
decoder_output_data = to_categorical(decoder_padded_seq_1[:,1:])

In [37]:
def data_splitter(encoder_input_data, decoder_input_data, decoder_output_data, test_size1=0.2, test_size2=0.3):
  
    en_train, en_test, de_train, de_test, out_train, out_test = train_test_split(encoder_input_data, decoder_input_data, decoder_output_data, test_size=test_size1)
    en_train, en_val, de_train, de_val, out_train, out_val = train_test_split(en_train, de_train, out_train, test_size=test_size2)
  
    return en_train, en_val, en_test, de_train, de_val, de_test, out_train, out_val, out_test

en_train, en_val, en_test, de_train, de_val, de_test, out_train, out_val, out_test = data_splitter(encoder_input_data, decoder_input_data, decoder_output_data)


KeyboardInterrupt: 

# Train Model

In [32]:
def seq2seq_model_builder(HIDDEN_DIM=300):
    
    encoder_inputs = Input(shape=(None,))
    x = Embedding(max_line_length+2, HIDDEN_DIM)(encoder_inputs)
    x, state_h, state_c = LSTM(HIDDEN_DIM, return_state=True)(x)
    encoder_states = [state_h, state_c]
    
    decoder_inputs = Input(shape=(None,))
    x = Embedding(max_line_length+2, HIDDEN_DIM)(decoder_inputs)
    x = LSTM(HIDDEN_DIM, return_sequences=True)(x, initial_state=encoder_states)
    
    
    # dense_layer = Dense(VOCAB_SIZE, activation='softmax')
    outputs = TimeDistributed(Dense(max_line_length+2, activation='softmax'))(x)
    model = Model([encoder_inputs, decoder_inputs], outputs)
    
    return model

In [33]:
model = seq2seq_model_builder(HIDDEN_DIM=300)
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    6000        input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    6000        input_2[0][0]                    
___________

In [36]:
%%time
batch_size = 128
epochs = 40

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
history = model.fit([en_train, de_train],out_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=([en_val, de_val],out_val))
score = model.evaluate([en_test, de_test],out_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

ValueError: Error when checking target: expected time_distributed_1 to have 3 dimensions, but got array with shape (31679, 20)

In [74]:
model.save('s2s.h5')

# Inference model

In [76]:
embeddings_index['eos']

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1.])

In [ ]:
embeddings_index.keys()

In [ ]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(HIDDEN_DIM,))
decoder_state_input_c = Input(shape=(HIDDEN_DIM,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Legacy stuff

In [ ]:
# Create a dictionary for the frequency of the vocabulary
vocab = {}
for question in short_questions:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for answer in short_answers:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
# Remove rare words from the vocabulary.
# We will aim to replace fewer than 5% of words with <UNK>
# You will see this ratio soon.
threshold = 10
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

In [ ]:
# In case we want to use a different vocabulary sizes for the source and target text, 
# we can set different threshold values.
# Nonetheless, we will create dictionaries to provide a unique integer for each word.
questions_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        questions_vocab_to_int[word] = word_num
        word_num += 1
        
answers_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        answers_vocab_to_int[word] = word_num
        word_num += 1

In [ ]:
# Add the unique tokens to the vocabulary dictionaries.
codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    questions_vocab_to_int[code] = len(questions_vocab_to_int)+1
    
for code in codes:
    answers_vocab_to_int[code] = len(answers_vocab_to_int)+1

In [ ]:
# Create dictionaries to map the unique integers to their respective words.
# i.e. an inverse dictionary for vocab_to_int.
questions_int_to_vocab = {v_i: v for v, v_i in questions_vocab_to_int.items()}
answers_int_to_vocab = {v_i: v for v, v_i in answers_vocab_to_int.items()}

In [ ]:
# Check the length of the dictionaries.
print(len(questions_vocab_to_int))
print(len(questions_int_to_vocab))
print(len(answers_vocab_to_int))
print(len(answers_int_to_vocab))

In [ ]:
# Add the end of sentence token to the end of every answer.
for i in range(len(short_answers)):
    short_answers[i] += ' <EOS>'

In [ ]:
# Convert the text to integers. 
# Replace any words that are not in the respective vocabulary with <UNK> 
questions_int = []
for question in short_questions:
    ints = []
    for word in question.split():
        if word not in questions_vocab_to_int:
            ints.append(questions_vocab_to_int['<UNK>'])
        else:
            ints.append(questions_vocab_to_int[word])
    questions_int.append(ints)
    
answers_int = []
for answer in short_answers:
    ints = []
    for word in answer.split():
        if word not in answers_vocab_to_int:
            ints.append(answers_vocab_to_int['<UNK>'])
        else:
            ints.append(answers_vocab_to_int[word])
    answers_int.append(ints)

In [ ]:
# Check the lengths
print(len(questions_int))
print(len(answers_int))

In [ ]:
# Calculate what percentage of all words have been replaced with <UNK>
word_count = 0
unk_count = 0

for question in questions_int:
    for word in question:
        if word == questions_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
for answer in answers_int:
    for word in answer:
        if word == answers_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
unk_ratio = round(unk_count/word_count,4)*100
    
print("Total number of words:", word_count)
print("Number of times <UNK> is used:", unk_count)
print("Percent of words that are <UNK>: {}%".format(round(unk_ratio,3)))

In [ ]:
# Sort questions and answers by the length of questions.
# This will reduce the amount of padding during training
# Which should speed up training and help to reduce the loss

sorted_questions = []
sorted_answers = []

for length in range(1, max_line_length+1):
    for i in enumerate(questions_int):
        if len(i[1]) == length:
            sorted_questions.append(questions_int[i[0]])
            sorted_answers.append(answers_int[i[0]])

print(len(sorted_questions))
print(len(sorted_answers))
print()
for i in range(3):
    print(sorted_questions[i])
    print(sorted_answers[i])
    print()

In [ ]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return input_data, targets, lr, keep_prob

In [ ]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [ ]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    '''Create the encoding layer'''
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = enc_cell,
                                                   cell_bw = enc_cell,
                                                   sequence_length = sequence_length,
                                                   inputs = rnn_inputs, 
                                                   dtype=tf.float32)
    return enc_state

In [ ]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob, batch_size):
    '''Decode the training data'''
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
            tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    train_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                     att_keys,
                                                                     att_vals,
                                                                     att_score_fn,
                                                                     att_construct_fn,
                                                                     name = "attn_dec_train")
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                              train_decoder_fn, 
                                                              dec_embed_input, 
                                                              sequence_length, 
                                                              scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)
    return output_fn(train_pred_drop)

In [ ]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob, batch_size):
    '''Decode the prediction data'''
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
            tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    infer_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_inference(output_fn, 
                                                                         encoder_state[0], 
                                                                         att_keys, 
                                                                         att_vals, 
                                                                         att_score_fn, 
                                                                         att_construct_fn, 
                                                                         dec_embeddings,
                                                                         start_of_sequence_id, 
                                                                         end_of_sequence_id, 
                                                                         maximum_length, 
                                                                         vocab_size, 
                                                                         name = "attn_dec_inf")
    infer_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                                infer_decoder_fn, 
                                                                scope=decoding_scope)
    
    return infer_logits

In [ ]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, vocab_to_int, keep_prob, batch_size):
    '''Create the decoding cell and input the parameters for the training and inference decoding layers'''
    
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        dec_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
        
        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_initializer()
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, 
                                                                vocab_size, 
                                                                None, 
                                                                scope=decoding_scope,
                                                                weights_initializer = weights,
                                                                biases_initializer = biases)

        train_logits = decoding_layer_train(encoder_state, 
                                            dec_cell, 
                                            dec_embed_input, 
                                            sequence_length, 
                                            decoding_scope, 
                                            output_fn, 
                                            keep_prob, 
                                            batch_size)
        decoding_scope.reuse_variables()
        infer_logits = decoding_layer_infer(encoder_state, 
                                            dec_cell, 
                                            dec_embeddings, 
                                            vocab_to_int['<GO>'],
                                            vocab_to_int['<EOS>'], 
                                            sequence_length - 1, 
                                            vocab_size,
                                            decoding_scope, 
                                            output_fn, keep_prob, 
                                            batch_size)

    return train_logits, infer_logits

In [ ]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, answers_vocab_size, 
                  questions_vocab_size, enc_embedding_size, dec_embedding_size, rnn_size, num_layers, 
                  questions_vocab_to_int):
    
    '''Use the previous functions to create the training and inference logits'''
    
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, 
                                                       answers_vocab_size+1, 
                                                       enc_embedding_size,
                                                       initializer = tf.random_uniform_initializer(0,1))
    enc_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob, sequence_length)

    dec_input = process_encoding_input(target_data, questions_vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([questions_vocab_size+1, dec_embedding_size], 0, 1))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    train_logits, infer_logits = decoding_layer(dec_embed_input, 
                                                dec_embeddings, 
                                                enc_state, 
                                                questions_vocab_size, 
                                                sequence_length, 
                                                rnn_size, 
                                                num_layers, 
                                                questions_vocab_to_int, 
                                                keep_prob, 
                                                batch_size)
    return train_logits, infer_logits

In [ ]:
# Set the Hyperparameters
epochs = 100
batch_size = 128
rnn_size = 512
num_layers = 2
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.005
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.75

In [ ]:
# Reset the graph to ensure that it is ready for training
tf.reset_default_graph()
# Start the session
sess = tf.InteractiveSession()
    
# Load the model inputs    
input_data, targets, lr, keep_prob = model_inputs()
# Sequence length will be the max line length for each batch
sequence_length = tf.placeholder_with_default(max_line_length, None, name='sequence_length')
# Find the shape of the input data for sequence_loss
input_shape = tf.shape(input_data)

# Create the training and inference logits
train_logits, inference_logits = seq2seq_model(
    tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(answers_vocab_to_int), 
    len(questions_vocab_to_int), encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, 
    questions_vocab_to_int)

# Create a tensor for the inference logits, needed if loading a checkpoint version of the model
tf.identity(inference_logits, 'logits')

with tf.name_scope("optimization"):
    # Loss function
    cost = tf.contrib.seq2seq.sequence_loss(
        train_logits,
        targets,
        tf.ones([input_shape[0], sequence_length]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

In [ ]:
indices = random.sample(range(len(conversation)), 50)
sample_context_list = []
sample_response_list = []

for index in indices:
    
    response = clean_text(conversation[index][-1])
        
    context = clean_text(conversation[index][0]) + "\n"
    for i in range(1, len(conversation[index]) - 1):
        
        if i % 2 == 0:
            prefix = "FS: "
        else:
            prefix = "SS: "
            
        context += clean_text(conversation[index][i]) + "\n"
        
    sample_context_list.append(context)
    sample_response_list.append(response)

#with open("cornell_movie_dialogue_sample.csv", "w") as handle:
#    for c, r in zip(sample_context_list, sample_response_list):
#        handle.write('"' + c + '"' + "#" + r + "\n")

In [ ]:
sample_context_list

In [ ]:
sample_response_list